<a href="https://colab.research.google.com/github/stacs-cp/permutation-classes-cp/blob/main/PP2022.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!source <(curl -s https://raw.githubusercontent.com/conjure-cp/conjure-notebook/main/installcolab.sh)
%load_ext conjure

Conjure: The Automated Constraint Modelling Tool
Release version 2.3.0
Repository version 987ee3fc3 (2022-06-10 21:50:17 +0100)


<IPython.core.display.Javascript object>

Conjure extension is loaded.
For usage help run: %conjure_help


### Model for Classical avoidance

* Find all permutations of a given length `length` which avoid the set of permutations `avoiding`.
* Each permutation is defined as an injective sequence of integers. The injectivity saves us from having to create a constraint that says the sequence has to contain all different values.
* The constraint for avoiding a permutation is (almost) as it is written on the box! 
* We actually find all permutations that contain the pattern, and then reject them.

In [2]:
%%conjure

letting length be 6
letting avoiding be {sequence(1,3,2,4)}

$ given length : int
$ given avoiding : set of sequence of int

find perm : sequence (size length, injective) of int(1..length)

such that
    forAll av in avoiding .
        !(exists ix : matrix indexed by [int(1..|av|)] of int(1..length) .
            (forAll i,j : int(1..|av|) . i < j -> ix[i] < ix[j]) /\
            (forAll n1, n2 : int(1..|av|) , n1 < n2 .
                av(n1) < av(n2) <-> perm(ix[n1]) < perm(ix[n2])))


{'perm': [1, 2, 3, 4, 5, 6]}

###Model for Mesh Avoidance
* All the above applies to Mesh Avoidance.
* We define the mesh to be a "relation" where the mesh is described as a pair of coordinates $\{0,\ldots,n\} \times \{0,\ldots,n\}$. 
* We create a "padded permutation" which is a matrix and includes the grid cells that are on the boundary.
* We follow yet again the definition of a mesh pattern.

In [3]:
%conjure_clear

Conjure model cleared


In [4]:
%%conjure --number-of-solutions=all

letting length be 4
letting avoiding be { ( sequence(1,3,2), relation((1,0),(1,2),(2,3),(3,0),(3,1)) ) }

$ given length : int
$ given avoiding : set of (sequence(injective) of int, relation of (int * int))

find perm : sequence (size length, injective) of int(1..length)

such that
    forAll (av,_) in avoiding .
        !(exists ix : matrix indexed by [int(1..|av|)] of int(1..length) .
            (forAll i,j : int(1..|av|) . i < j -> ix[i] < ix[j]) /\
            (forAll n1, n2 : int(1..|av|) , n1 < n2 .
                av(n1) < av(n2) <-> perm(ix[n1]) < perm(ix[n2])))

$ creating a padded version of perm, where position 0 contains the value 0 and position length+1 contains the value length+1
$ this is only used for mesh avoidance/containment
find permPadded : matrix indexed by [int(0..length+1)] of int(0..length+1)
such that permPadded[0] = 0, permPadded[length+1] = length+1
such that forAll i : int(1..length) . perm(i) = permPadded[i]

such that
    $ av is the pattern, mesh is the mesh as a relation
    forAll (av, mesh) in avoiding .
    $ Build the inverse of 'av'. This is completely evaluated before solving.
    exists avinv: matrix indexed by [int(0..|av|+1)] of int(0..|av|+1),
                avinv[0] = 0 /\ avinv[|av|+1] = |av|+1 /\
                (forAll i: int(1..|av|) . avinv[av(i)] = i).
        $ Look for all places where the pattern can occur
        forAll ix : matrix indexed by [int(0..|av|+1)] of int(0..length+1),
            and([ ix[0]=0
                , ix[|av|+1]=length+1
                , forAll i : int(0..|av|) . ix[i] < ix[i+1]
                , forAll n1, n2 : int(1..|av|) , n1 < n2 .
                    av(n1) < av(n2) <-> permPadded[ix[n1]] < permPadded[ix[n2]]
                ]) .
            (
                $ If we find the pattern, make sure there is at least one value in some cell of the mesh
                exists (i,j) in mesh.
                    exists z: int(ix[i]+1..ix[i+1]-1). (permPadded[ix[avinv[j]]] <= permPadded[z] /\ permPadded[z] <= permPadded[ix[avinv[j+1]]])
            )

{'conjure_solutions': [{'perm': [4, 2, 3, 1],
   'permPadded': {'0': 0, '1': 4, '2': 2, '3': 3, '4': 1, '5': 5}},
  {'perm': [2, 3, 1, 4],
   'permPadded': {'0': 0, '1': 2, '2': 3, '3': 1, '4': 4, '5': 5}},
  {'perm': [2, 1, 3, 4],
   'permPadded': {'0': 0, '1': 2, '2': 1, '3': 3, '4': 4, '5': 5}},
  {'perm': [3, 2, 4, 1],
   'permPadded': {'0': 0, '1': 3, '2': 2, '3': 4, '4': 1, '5': 5}},
  {'perm': [4, 2, 1, 3],
   'permPadded': {'0': 0, '1': 4, '2': 2, '3': 1, '4': 3, '5': 5}},
  {'perm': [4, 1, 2, 3],
   'permPadded': {'0': 0, '1': 4, '2': 1, '3': 2, '4': 3, '5': 5}},
  {'perm': [3, 4, 2, 1],
   'permPadded': {'0': 0, '1': 3, '2': 4, '3': 2, '4': 1, '5': 5}},
  {'perm': [4, 3, 1, 2],
   'permPadded': {'0': 0, '1': 4, '2': 3, '3': 1, '4': 2, '5': 5}},
  {'perm': [1, 2, 3, 4],
   'permPadded': {'0': 0, '1': 1, '2': 2, '3': 3, '4': 4, '5': 5}},
  {'perm': [3, 2, 1, 4],
   'permPadded': {'0': 0, '1': 3, '2': 2, '3': 1, '4': 4, '5': 5}},
  {'perm': [2, 3, 4, 1],
   'permPadded': {'0': 0

###Simple Permutations
* Again the model is based on the defintion.


In [5]:
%conjure_clear

Conjure model cleared


In [6]:
%%conjure

letting length be 4
find perm : sequence (size length, injective) of int(1..length)

such that
    [ max(subperm) - min(subperm) + 1 != |subperm|          $ the values are not contiguous
    | i : int(1..length)                                    $ start index of the sub perm
    , j : int(1..length)                                    $ end index of the sub perm
    , i < j                                                 $ start comes before end
    , (i,j) != (1,length)                                   $ it is not the full permutation
    , letting subperm be [perm(k) | k : int(i..j)]          $ give the sub perm a name
    ]

{'perm': [2, 4, 1, 3]}

Let us just add this to the mesh avoiding permutations

In [7]:
%conjure_clear

Conjure model cleared


In [8]:
%%conjure --number-of-solutions=all

letting length be 5
letting avoiding be { ( sequence(1,3,2), relation((1,0),(1,2),(2,3),(3,0),(3,1)) ) }

$ given length : int
$ given avoiding : set of (sequence(injective) of int, relation of (int * int))

find perm : sequence (size length, injective) of int(1..length)
find permPadded : matrix indexed by [int(0..length+1)] of int(0..length+1)

such that permPadded[0] = 0, permPadded[length+1] = length+1

such that forAll i : int(1..length) . perm(i) = permPadded[i]

such that
    forAll (av, mesh) in avoiding .
    exists avinv: matrix indexed by [int(0..|av|+1)] of int(0..|av|+1),
                avinv[0] = 0 /\ avinv[|av|+1] = |av|+1 /\
                (forAll i: int(1..|av|) . avinv[av(i)] = i).
        forAll ix : matrix indexed by [int(0..|av|+1)] of int(0..length+1),
            and([ ix[0]=0
                , ix[|av|+1]=length+1
                , forAll i : int(0..|av|) . ix[i] < ix[i+1]
                , forAll n1, n2 : int(1..|av|) , n1 < n2 .
                    av(n1) < av(n2) <-> permPadded[ix[n1]] < permPadded[ix[n2]]
                ]) .
            (
                exists (i,j) in mesh.
                    exists z: int(ix[i]+1..ix[i+1]-1). (permPadded[ix[avinv[j]]] <= permPadded[z] /\ permPadded[z] <= permPadded[ix[avinv[j+1]]])
            )

such that
    [ max(subperm) - min(subperm) + 1 != |subperm|
    | i : int(1..length)
    , j : int(1..length)
    , i < j
    , (i,j) != (1,length)
    , letting subperm be [perm(k) | k : int(i..j)]
    ]


{'perm': [4, 1, 3, 5, 2],
 'permPadded': {'0': 0, '1': 4, '2': 1, '3': 3, '4': 5, '5': 2, '6': 6}}

###Inversions
* This model will just tell us the number of inversions a permutation contains.
* We have to have a bounded domain, so we give it a "very loose" upper bound.

In [9]:
%conjure_clear

Conjure model cleared


In [10]:
%%conjure
letting length be 6
find perm : sequence (size length, injective) of int(1..length)

find inversionCount : int(0..length**2)

such that
    inversionCount =
        sum([ 1
            | i,j : int(1..length)
            , i < j
            , perm(i) > perm(j)
            ])


{'inversionCount': 0, 'perm': [1, 2, 3, 4, 5, 6]}

* Let us add this into model that avoids mesh patterns, and looks for simple permutations.
* **but** let us look for a given number of inversions.

In [11]:
%conjure_clear

Conjure model cleared


In [12]:
%%conjure --number-of-solutions=all

letting length be 5
letting avoiding be { ( sequence(1,3,2), relation((1,0),(1,2),(2,3),(3,0),(3,1)) ) }

find perm : sequence (size length, injective) of int(1..length)
find permPadded : matrix indexed by [int(0..length+1)] of int(0..length+1)

such that permPadded[0] = 0, permPadded[length+1] = length+1

such that forAll i : int(1..length) . perm(i) = permPadded[i]

such that
    forAll (av, mesh) in avoiding .
    exists avinv: matrix indexed by [int(0..|av|+1)] of int(0..|av|+1),
                avinv[0] = 0 /\ avinv[|av|+1] = |av|+1 /\
                (forAll i: int(1..|av|) . avinv[av(i)] = i).
        forAll ix : matrix indexed by [int(0..|av|+1)] of int(0..length+1),
            and([ ix[0]=0
                , ix[|av|+1]=length+1
                , forAll i : int(0..|av|) . ix[i] < ix[i+1]
                , forAll n1, n2 : int(1..|av|) , n1 < n2 .
                    av(n1) < av(n2) <-> permPadded[ix[n1]] < permPadded[ix[n2]]
                ]) .
            (
                exists (i,j) in mesh.
                    exists z: int(ix[i]+1..ix[i+1]-1). (permPadded[ix[avinv[j]]] <= permPadded[z] /\ permPadded[z] <= permPadded[ix[avinv[j+1]]])
            )

such that
    [ max(subperm) - min(subperm) + 1 != |subperm|
    | i : int(1..length)
    , j : int(1..length)
    , i < j
    , (i,j) != (1,length)
    , letting subperm be [perm(k) | k : int(i..j)]
    ]

letting inversionCount be 4

such that
    inversionCount =
        sum([ 1
            | i,j : int(1..length)
            , i < j
            , perm(i) > perm(j)
            ])

Exception: No solution found for this Essence model.


###Pattern Containment
What about... if we avoid a set of patterns but know that a set of patterns are definitely always contained?

In [13]:
%conjure_clear

Conjure model cleared


In [14]:
%%conjure


letting length be 5
letting containing be { ( sequence(1,3,2), relation((3,3)) ) }
letting avoiding be { ( sequence(1,3,2), relation((1,0),(1,2),(2,3),(3,0),(3,1)) ) }

find perm : sequence (size length, injective) of int(1..length)


$ creating a padded version of perm, where position 0 contains the value 0 and position length+1 contains the value length+1
$ this is only used for mesh avoidance/containment
find permPadded : matrix indexed by [int(0..length+1)] of int(0..length+1)
such that permPadded[0] = 0, permPadded[length+1] = length+1
such that forAll i : int(1..length) . perm(i) = permPadded[i]

such that
    $ av is the pattern, mesh is the mesh as a relation
    forAll (cont, mesh) in containing .
    $ Build the inverse of 'av'. This is completely evaluated before solving.
    exists continv: matrix indexed by [int(0..|cont|+1)] of int(0..|cont|+1),
                continv[0] = 0 /\ continv[|cont|+1] = |cont|+1 /\
                (forAll i: int(1..|cont|) . continv[cont(i)] = i).
        $ Look for all places where the pattern can occur
        exists ix : matrix indexed by [int(0..|cont|+1)] of int(0..length+1),
            and([ ix[0]=0
                , ix[|cont|+1]=length+1
                , forAll i : int(0..|cont|) . ix[i] < ix[i+1]
                , forAll n1, n2 : int(1..|cont|) , n1 < n2 .
                    cont(n1) < cont(n2) <-> permPadded[ix[n1]] < permPadded[ix[n2]]
                ]) .
            !(
                $ If we find the pattern, make sure there is at least one value in some cell of the mesh
                exists (i,j) in mesh.
                    exists z: int(ix[i]+1..ix[i+1]-1). (permPadded[ix[continv[j]]] <= permPadded[z] /\ permPadded[z] <= permPadded[ix[continv[j+1]]])
            )

such that
    forAll (av, mesh) in avoiding .
    exists avinv: matrix indexed by [int(0..|av|+1)] of int(0..|av|+1),
                avinv[0] = 0 /\ avinv[|av|+1] = |av|+1 /\
                (forAll i: int(1..|av|) . avinv[av(i)] = i).
        forAll ix : matrix indexed by [int(0..|av|+1)] of int(0..length+1),
            and([ ix[0]=0
                , ix[|av|+1]=length+1
                , forAll i : int(0..|av|) . ix[i] < ix[i+1]
                , forAll n1, n2 : int(1..|av|) , n1 < n2 .
                    av(n1) < av(n2) <-> permPadded[ix[n1]] < permPadded[ix[n2]]
                ]) .
            (
                exists (i,j) in mesh.
                    exists z: int(ix[i]+1..ix[i+1]-1). (permPadded[ix[avinv[j]]] <= permPadded[z] /\ permPadded[z] <= permPadded[ix[avinv[j+1]]])
            )



{'perm': [1, 2, 4, 5, 3],
 'permPadded': {'0': 0, '1': 1, '2': 2, '3': 4, '4': 5, '5': 3, '6': 6}}